Go to Runtime --> change runtime type and select TPU.
Then, run the below commands, and restart the kernal to reflect changes.


In [ ]:
#!pip install accelerate -U
#!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import torch

# Set a random seed for CPU
torch.manual_seed(42)

# Set a random seed for GPU
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

# Other random initializations
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

BERT

In [ ]:
import pandas as pd
import json

with open('./annotated_data-train.json', 'r') as f:
    data = json.load(f)


df1 = pd.DataFrame(data)

with open('./annotated_data-test.json', 'r') as f:
    data = json.load(f)

df2 = pd.DataFrame(data)

df=pd.concat([df1,df2])

df=df[['text','label']]

label_map = {'noHate': 0, 'hate': 1}

# Use the map function to apply the mapping
df['label_value'] = df['label'].map(label_map)

df.reset_index(drop=True, inplace=True)

data = df[['text','label_value']]

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

model = model.to('cuda')

X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

X_train_tokenized.keys()

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)



def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer

from transformers import Trainer
from transformers import TrainingArguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.3400350511074066,
 'eval_accuracy': 0.8705636743215032,
 'eval_precision': 0.8647540983606558,
 'eval_recall': 0.8791666666666667,
 'eval_f1': 0.871900826446281,
 'eval_runtime': 15.9861,
 'eval_samples_per_second': 29.964,
 'eval_steps_per_second': 3.753,
 'epoch': 1.0}

In [ ]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F

df=pd.read_csv('MainDataset.csv')



df=df[['text','label']]

label_map = {0: 0, 1: 1}

# Use the map function to apply the mapping
df['label_value'] = df['label'].map(label_map)

df.reset_index(drop=True, inplace=True)

data = df[['text','label_value']]

# Initialize BERT tokenizer and model




X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

X_train_tokenized.keys()

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])



val_dataset_m = Dataset(X_val_tokenized, y_val)

In [ ]:
trainer.evaluate(eval_dataset=val_dataset_m)

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.7449175119400024,
 'eval_accuracy': 0.6623164763458401,
 'eval_precision': 0.7980769230769231,
 'eval_recall': 0.5015105740181269,
 'eval_f1': 0.6159554730983302,
 'eval_runtime': 12.3059,
 'eval_samples_per_second': 49.814,
 'eval_steps_per_second': 6.257,
 'epoch': 1.0}

In [ ]:
torch.save(model.state_dict(), 'bert_model.pth')


loaded_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
loaded_model.load_state_dict(torch.load('bert_model.pth'))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
import pandas as pd
import csv

df=pd.read_csv('SuicidalThoughts.csv')



df=df[['text','label']]

label_map = {'non-suicide': 0, 'suicide': 1}

# Use the map function to apply the mapping
df['label_value'] = df['label'].map(label_map)

df.reset_index(drop=True, inplace=True)

data = df[['text','label_value']]


model = loaded_model.to('cuda')

X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

X_train_tokenized.keys()

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)



def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer

from transformers import Trainer
from transformers import TrainingArguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.evaluate()

Step,Training Loss


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.25358328223228455,
 'eval_accuracy': 0.9283333333333333,
 'eval_precision': 0.9610983981693364,
 'eval_recall': 0.9417040358744395,
 'eval_f1': 0.9513023782559457,
 'eval_runtime': 19.8687,
 'eval_samples_per_second': 30.198,
 'eval_steps_per_second': 3.775,
 'epoch': 1.0}

In [ ]:
trainer.evaluate(eval_dataset=val_dataset_m)

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 1.798611044883728,
 'eval_accuracy': 0.5432300163132137,
 'eval_precision': 0.8493150684931506,
 'eval_recall': 0.18731117824773413,
 'eval_f1': 0.3069306930693069,
 'eval_runtime': 12.4821,
 'eval_samples_per_second': 49.11,
 'eval_steps_per_second': 6.169,
 'epoch': 1.0}

In [ ]:
torch.save(model.state_dict(), 'bert_model2.pth')


loaded_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
loaded_model.load_state_dict(torch.load('bert_model2.pth'))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
import pandas as pd
import csv

df=pd.read_csv('TerroristStmtsCSV.csv')



df=df[['text','label']]

label_map = {0: 0, 1: 1}

# Use the map function to apply the mapping
df['label_value'] = df['label'].map(label_map)

df.reset_index(drop=True, inplace=True)

data = df[['text','label_value']]


model = loaded_model.to('cuda')

X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

X_train_tokenized.keys()

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)



def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer

from transformers import Trainer
from transformers import TrainingArguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.evaluate()

Step,Training Loss


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.021443389356136322,
 'eval_accuracy': 0.9924242424242424,
 'eval_precision': 1.0,
 'eval_recall': 0.9850746268656716,
 'eval_f1': 0.9924812030075187,
 'eval_runtime': 3.7406,
 'eval_samples_per_second': 70.577,
 'eval_steps_per_second': 8.822,
 'epoch': 1.0}

In [ ]:
trainer.evaluate(eval_dataset=val_dataset_m)

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.9153760075569153,
 'eval_accuracy': 0.8140293637846656,
 'eval_precision': 0.8974358974358975,
 'eval_recall': 0.7401812688821753,
 'eval_f1': 0.8112582781456954,
 'eval_runtime': 12.4304,
 'eval_samples_per_second': 49.315,
 'eval_steps_per_second': 6.194,
 'epoch': 1.0}

In [ ]:
torch.save(model.state_dict(), 'bert_model3.pth')


loaded_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
loaded_model.load_state_dict(torch.load('bert_model3.pth'))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
import pandas as pd
import csv

df=pd.read_csv('MainDataset.csv')



df=df[['text','label']]

label_map = {0: 0, 1: 1}

# Use the map function to apply the mapping
df['label_value'] = df['label'].map(label_map)

df.reset_index(drop=True, inplace=True)

data = df[['text','label_value']]


loaded_model = loaded_model.to('cuda')

X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

X_train_tokenized.keys()

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)



def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer

from transformers import Trainer
from transformers import TrainingArguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.evaluate()

Step,Training Loss


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.06699677556753159,
 'eval_accuracy': 0.9804241435562806,
 'eval_precision': 0.9818731117824774,
 'eval_recall': 0.9818731117824774,
 'eval_f1': 0.9818731117824774,
 'eval_runtime': 12.4516,
 'eval_samples_per_second': 49.23,
 'eval_steps_per_second': 6.184,
 'epoch': 1.0}

RoBERTa

In [ ]:
import pandas as pd
import csv

df=pd.read_csv('MainDataset.csv')



df=df[['text','label']]

label_map = {0: 0, 1: 1}

# Use the map function to apply the mapping
df['label_value'] = df['label'].map(label_map)

df.reset_index(drop=True, inplace=True)

data = df[['text','label_value']]

# Initialize BERT tokenizer and model




X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

X_train_tokenized.keys()

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])



val_dataset_m = Dataset(X_val_tokenized, y_val)

In [ ]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Load data from JSON files
with open('./annotated_data-train.json', 'r') as f:
    train_data = json.load(f)

with open('./annotated_data-test.json', 'r') as f:
    test_data = json.load(f)

# Concatenate train and test dataframes
df = pd.concat([pd.DataFrame(train_data), pd.DataFrame(test_data)])

# Select 'text' and 'label' columns
df = df[['text', 'label']]

# Map label values to integers
label_map = {'noHate': 0, 'hate': 1}
df['label_value'] = df['label'].map(label_map)

# Reset index
df.reset_index(drop=True, inplace=True)

data = df[['text', 'label_value']]

# Initialize RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Split data into train and validation sets
X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

# Tokenize train and validation sets
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

# Define Dataset class
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

# Create train and validation datasets
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Define Trainer arguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8
)

# Use the Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


{'eval_loss': 0.44277119636535645,
 'eval_accuracy': 0.824634655532359,
 'eval_precision': 0.8070866141732284,
 'eval_recall': 0.8541666666666666,
 'eval_f1': 0.8299595141700405,
 'eval_runtime': 13.2548,
 'eval_samples_per_second': 36.138,
 'eval_steps_per_second': 4.527,
 'epoch': 1.0}

In [ ]:
trainer.evaluate(eval_dataset=val_dataset_m)

{'eval_loss': 1.1551337242126465,
 'eval_accuracy': 0.49429037520391517,
 'eval_precision': 0.8181818181818182,
 'eval_recall': 0.08157099697885196,
 'eval_f1': 0.14835164835164838,
 'eval_runtime': 12.4808,
 'eval_samples_per_second': 49.115,
 'eval_steps_per_second': 6.169,
 'epoch': 1.0}

In [ ]:
torch.save(model.state_dict(), 'roberta_model.pth')


loaded_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

loaded_model.load_state_dict(torch.load('roberta_model.pth'))


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
import pandas as pd
import csv

df=pd.read_csv('SuicidalThoughts.csv')



df=df[['text','label']]

label_map = {'non-suicide': 0, 'suicide': 1}

# Use the map function to apply the mapping
df['label_value'] = df['label'].map(label_map)

df.reset_index(drop=True, inplace=True)

data = df[['text','label_value']]


model = loaded_model.to('cuda')

X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

X_train_tokenized.keys()

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)



def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer

from transformers import Trainer
from transformers import TrainingArguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.evaluate()

Step,Training Loss


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.21586397290229797,
 'eval_accuracy': 0.95,
 'eval_precision': 0.9622222222222222,
 'eval_recall': 0.9708520179372198,
 'eval_f1': 0.9665178571428571,
 'eval_runtime': 18.0835,
 'eval_samples_per_second': 33.179,
 'eval_steps_per_second': 4.147,
 'epoch': 1.0}

In [ ]:
trainer.evaluate(eval_dataset=val_dataset_m)

<class 'transformers.trainer_utils.EvalPrediction'>


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 2.4764246940612793,
 'eval_accuracy': 0.4600326264274062,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_f1': 0.0,
 'eval_runtime': 12.4005,
 'eval_samples_per_second': 49.434,
 'eval_steps_per_second': 6.209,
 'epoch': 1.0}

In [ ]:
torch.save(model.state_dict(), 'roberta_model2.pth')


loaded_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
loaded_model.load_state_dict(torch.load('roberta_model2.pth'))


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
import pandas as pd
import csv

df=pd.read_csv('TerroristStmtsCSV.csv')



df=df[['text','label']]

label_map = {0: 0, 1: 1}

# Use the map function to apply the mapping
df['label_value'] = df['label'].map(label_map)

df.reset_index(drop=True, inplace=True)

data = df[['text','label_value']]


model = loaded_model.to('cuda')

X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

X_train_tokenized.keys()

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)



def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer

from transformers import Trainer
from transformers import TrainingArguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.evaluate()

Step,Training Loss


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.0006570756086148322,
 'eval_accuracy': 1.0,
 'eval_precision': 1.0,
 'eval_recall': 1.0,
 'eval_f1': 1.0,
 'eval_runtime': 3.564,
 'eval_samples_per_second': 74.075,
 'eval_steps_per_second': 9.259,
 'epoch': 1.0}

In [ ]:
trainer.evaluate(eval_dataset=val_dataset_m)

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 3.377516746520996,
 'eval_accuracy': 0.5399673735725938,
 'eval_precision': 0.5399673735725938,
 'eval_recall': 1.0,
 'eval_f1': 0.701271186440678,
 'eval_runtime': 12.3773,
 'eval_samples_per_second': 49.526,
 'eval_steps_per_second': 6.221,
 'epoch': 1.0}

In [ ]:
torch.save(model.state_dict(), 'roberta_model3.pth')


loaded_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
loaded_model.load_state_dict(torch.load('roberta_model3.pth'))


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
import pandas as pd
import csv

df=pd.read_csv('MainDataset.csv')



df=df[['text','label']]

label_map = {0: 0, 1: 1}

# Use the map function to apply the mapping
df['label_value'] = df['label'].map(label_map)

df.reset_index(drop=True, inplace=True)

data = df[['text','label_value']]


loaded_model = loaded_model.to('cuda')

X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

X_train_tokenized.keys()

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)



def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer

from transformers import Trainer
from transformers import TrainingArguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.evaluate()

Step,Training Loss


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.10126999765634537,
 'eval_accuracy': 0.9804241435562806,
 'eval_precision': 0.9877675840978594,
 'eval_recall': 0.9758308157099698,
 'eval_f1': 0.9817629179331306,
 'eval_runtime': 15.4623,
 'eval_samples_per_second': 39.645,
 'eval_steps_per_second': 4.98,
 'epoch': 1.0}

In [ ]:
!pip install accelerate -U

Baseline for BERT

In [ ]:
import pandas as pd
import csv

df=pd.read_csv('MainDataset.csv')



df=df[['text','label']]

label_map = {0: 0, 1: 1}

# Use the map function to apply the mapping
df['label_value'] = df['label'].map(label_map)

df.reset_index(drop=True, inplace=True)

data = df[['text','label_value']]

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

model = model.to('cuda')

X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

X_train_tokenized.keys()

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)



def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer

from transformers import Trainer
from transformers import TrainingArguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.1034141257405281,
 'eval_accuracy': 0.9706362153344209,
 'eval_precision': 0.9785932721712538,
 'eval_recall': 0.9667673716012085,
 'eval_f1': 0.9726443768996961,
 'eval_runtime': 14.0171,
 'eval_samples_per_second': 43.732,
 'eval_steps_per_second': 5.493,
 'epoch': 1.0}

In [ ]:
predictions = trainer.predict(val_dataset)

<class 'transformers.trainer_utils.EvalPrediction'>


In [ ]:
import pandas as pd

preds = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

results_df = pd.DataFrame({'text': X_val, 'true_label': true_labels, 'predicted_label': preds})

misclassified_df = results_df[results_df['true_label'] != results_df['predicted_label']]

misclassified_df.to_csv('misclassified_instances.csv', index=False)


Baseline for RoBERTa

In [ ]:
import pandas as pd
import csv

df=pd.read_csv('MainDataset.csv')



df=df[['text','label']]

label_map = {0: 0, 1: 1}

# Use the map function to apply the mapping
df['label_value'] = df['label'].map(label_map)

df.reset_index(drop=True, inplace=True)

data = df[['text','label_value']]

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import RobertaTokenizer, RobertaForSequenceClassification

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

model = model.to('cuda')

X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

X_train_tokenized.keys()

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)



def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer

from transformers import Trainer
from transformers import TrainingArguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.evaluate()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.11351493746042252,
 'eval_accuracy': 0.9738988580750407,
 'eval_precision': 0.9876160990712074,
 'eval_recall': 0.9637462235649547,
 'eval_f1': 0.9755351681957186,
 'eval_runtime': 15.0237,
 'eval_samples_per_second': 40.802,
 'eval_steps_per_second': 5.125,
 'epoch': 1.0}